In [2]:
import numpy as np
import glob
import pandas
from bokeh.io import *
from bokeh.plotting import figure, show, output_file, ColumnDataSource

df0 = pandas.DataFrame()
for f in glob.glob("C:\\Users\\aelek\\source\\repos\\FantaSeaViz\\data\\2017-18\\gws\\*"):
    dftmp = pandas.read_csv(f, encoding='ansi')   
    dftmp['name'] = dftmp['name'].str.replace('_', ' ')
    df0 = df0.append(dftmp)

# add cleaned data - df1
df1 = pandas.read_csv("C:\\Users\\aelek\\source\\repos\\FantaSeaViz\\data\\2017-18\\cleaned_players.csv", encoding='utf8')
df1['name_lower'] = df1['first_name'].str.lower() + ' ' + df1['second_name'].str.lower()
df1 = df1.set_index('name_lower')
df1 = df1.sort_values(['name_lower'], ascending=True)

# group by player and calculate ratio - df2
df2 = df0.groupby(['name']).median()
df2['value'] = df2['value']/10
df2['vpc_ratio'] = df2['total_points'] / df2['value']
df2 = df2[(df2['total_points'] > 0) & (df2['vpc_ratio'] > 0)]
df2['name'] = df2.index.values
df2['name_lower'] = df2['name'].str.lower()
df2 = df2.set_index('name_lower')
df2 = df2.sort_values(['name_lower'], ascending=True)


# merge
df = pandas.merge(df1, df2, on='name_lower', how='outer')
df.to_csv('out.csv', encoding='utf8')

In [3]:
# set data source for visual
source = ColumnDataSource(data=dict(
    x=df['bps_x']/10,
    y=df['total_points_x'],
    names=df['name'],
    vpc_ratio=df['vpc_ratio'],
    circle_size=df['now_cost']/50,
    fill_alpha = np.full((len(df['name']), 1), 0.6)))

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)
p.hover.tooltips = [
    ("name", "@names"),
    ("bps", "@x"),
    ("total points", "@y"),
    ("value per cost", "@vpc_ratio")
]

p.scatter(x='x', y='y', radius='circle_size', fill_alpha='fill_alpha', source=source)

output_file('C:/users/aelek/Desktop/hazard.html')
show(p)

In [ ]:
from IPython.display import display, Markdown
display(Markdown("![player](https://platform-static-files.s3.amazonaws.com/premierleague/photos/players/110x140/p11334.png)"))


![player](https://platform-static-files.s3.amazonaws.com/premierleague/photos/players/110x140/p11334.png)